In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ || 
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data. 
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to 
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. 
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.




In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we 
continue to use the CPU. 



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and 
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method. 



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print 
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_. 
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print("logits: ", logits)
pred_probab = nn.Softmax(dim=1)(logits)
print("pred_probab: ", pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

logits:  tensor([[0.0000, 0.0448, 0.0000, 0.0000, 0.0441, 0.0823, 0.0000, 0.0306, 0.0502,
         0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
pred_probab:  tensor([[0.0975, 0.1019, 0.0975, 0.0975, 0.1019, 0.1058, 0.0975, 0.1005, 0.1025,
         0.0975]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([5], device='cuda:0')


Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we 
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as 
we pass it through the network. 



In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_ 
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear 
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2468, -0.1030,  0.1281, -0.3298,  0.2881,  0.3125,  0.1464, -0.1423,
          0.2314,  0.1264,  0.0531,  0.2588,  0.1408,  0.4403, -0.5555,  0.1551,
         -0.2131, -0.0737,  0.2948, -0.1597],
        [-0.0272, -0.1865,  0.5147,  0.0309,  0.2483,  0.1361,  0.0889,  0.0275,
         -0.1126,  0.0082, -0.0512,  0.2233, -0.0101,  0.2654, -0.2268,  0.3292,
         -0.0817,  0.2794,  0.4879, -0.0716],
        [ 0.0276, -0.2535,  0.2335, -0.1023,  0.0385,  0.1427, -0.4406, -0.0654,
         -0.2634,  0.3240, -0.1565,  0.2286,  0.0695,  0.4258, -0.1342,  0.0573,
          0.1497,  0.5257,  0.3109, -0.3548]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2468, 0.0000, 0.1281, 0.0000, 0.2881, 0.3125, 0.1464, 0.0000, 0.2314,
         0.1264, 0.0531, 0.2588, 0.1408, 0.4403, 0.0000, 0.1551, 0.0000, 0.0000,
         0.2948, 0.0000],
        [0.0000, 0.0000, 0.5147, 0.0309, 0.2483, 0.1361, 0.0889, 0.0275, 0.0000,
         0.0082, 0.0000, 0.2233, 0.0000, 0.2654, 0.000

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values 
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along 
which the values must sum to 1. 



In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing ``nn.Module`` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's ``parameters()`` or ``named_parameters()`` methods. 

In this example, we iterate over each parameter, and print its size and a preview of its values. 




In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0354,  0.0181,  0.0058,  ..., -0.0252, -0.0135, -0.0310],
        [-0.0130,  0.0342, -0.0231,  ..., -0.0262,  0.0231,  0.0239]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0177, 0.0026], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0039, -0.0360, -0.0411,  ..., -0.0032, -0.0149, -0.0415],
        [-0.0338,  0.0086, -0.0165,  ...,  0.0432,  0.0358, -0.0258]],
       device='cuda:0

Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

